In [ ]:
### =================================================================== ###
### KODE LENGKAP FINAL (TERMASUK PERBAIKAN DATA AUGMENTATION)           ###
### =================================================================== ###

# BAGIAN 1: IMPOR SEMUA LIBRARY YANG DIBUTUHKAN
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np
import time
import copy
import os

print("--- [TAHAP 1] MEMPERSIAPKAN DATA ---")

# Definisikan Path dan Transformasi
full_data_path = '/kaggle/input/padangfood/dataset_padang_food/'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 1. Muat dataset SATU KALI tanpa transformasi, hanya untuk mendapatkan label untuk stratifikasi
dataset_for_splitting = datasets.ImageFolder(full_data_path)

# 2. Dapatkan indeks untuk train dan val
train_idx, val_idx = train_test_split(
    np.arange(len(dataset_for_splitting)),
    test_size=0.2,
    random_state=42,
    stratify=dataset_for_splitting.targets
)

# 3. Buat DUA objek dataset TERPISAH, masing-masing dengan transformasinya sendiri
train_transformed_dataset = datasets.ImageFolder(full_data_path, transform=data_transforms['train'])
val_transformed_dataset = datasets.ImageFolder(full_data_path, transform=data_transforms['val'])

# 4. Buat Subset dari dataset yang sudah ditransformasi dengan benar
train_dataset = Subset(train_transformed_dataset, train_idx)
val_dataset = Subset(val_transformed_dataset, val_idx)

# 5. Buat DataLoaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
}

# 6. Dapatkan Ukuran Dataset dan Nama Kelas
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = dataset_for_splitting.classes 
print(f"Data berhasil disiapkan. Total kelas: {len(class_names)}.")
print(f"Data training: {dataset_sizes['train']}, Data validasi: {dataset_sizes['val']}")


print("\n--- [TAHAP 2] MEMPERSIAPKAN MODEL DAN TRAINING ---")

# Definisikan Model
model = models.resnet18(weights='IMAGENET1K_V1')
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# Siapkan Komponen Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
print("Model, criterion, dan optimizer siap.")

# Definisikan Fungsi Training
def train_model(model, criterion, optimizer, num_epochs=15):
    start_time = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - start_time
    print(f'Training selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Akurasi validasi terbaik: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

print("Fungsi training siap.")

print("\n--- [TAHAP 3] MEMULAI TRAINING ---")
# Jalankan Training
model_trained = train_model(model, criterion, optimizer, num_epochs=15)

# Simpan Model
torch.save(model_trained.state_dict(), 'naspad_classifier.pt')
print("\nProses selesai! Model dengan akurasi terbaik sudah disimpan.")